<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/cleanBorderData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:

# Panel Data
import pandas as pd
# # System folders
import os
from pathlib import Path

dataPath = Path.joinpath(Path.cwd(),'data','borderCrossing.csv')
df = pd.read_csv(dataPath)
df.sample(3)

# read in the data
# url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
# df = pd.read_csv(url)
# df.sample(3)


,Unnamed: 0,Port Name,State,Port Code,Border,Date,Measure,Value,Location
9808,30151,Del Bonita,Montana,3322,US-Canada Border,4/1/2016 0:00,Train Passengers,0,POINT (-112.79 49)
27260,82539,Point Roberts,Washington,3017,US-Canada Border,1/1/2013 0:00,Personal Vehicle Passengers,109723,POINT (-123.07000000000001 49)
55586,167469,Fort Kent,Maine,110,US-Canada Border,9/1/2007 0:00,Pedestrians,1364,POINT (-68.6 47.25)


In [43]:
df['Measure'].value_counts()

Personal Vehicle Passengers    30196
Bus Passengers                 28820
Pedestrians                    28697
Train Passengers               27623
Name: Measure, dtype: int64

In [44]:
# Only interested in Passenger / Pedestrian crossings
people = df['Measure'].str.contains('Passengers|Pedestrians', case = False)

In [45]:
# Check to see how much data frame shrinks after filtering down
print(f'All measures: {df.shape}')
#df = df[people] # This gave index error when re-running, changed to df.loc
df = df.loc[people]
print(f'Just people: {df.shape}')

All measures: (115336, 9)
Just people: (115336, 9)


In [46]:
# Break out Location into latitude and longitude
                # substring from 'POINT( ' to ')' and split on ' ' grabbing first then second element set as float
df['Latitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[1].astype(float)
df['Longitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[0].astype(float)

In [47]:
# Drop unnecessary columns 
for col in df.columns:
    if col in ['Port Code','Port Name','Location','Unnamed: 0','index']:
        df.drop([col], axis=1, inplace=True)
print(df.shape)

(115336, 7)


In [48]:
# Need state abbreviations not full names
df['Abrv'] = df['State'].map({'Alaska':'AL',
                               'Arizona':'AZ',
                               'California':'CA',
                               'Idaho':'ID',
                               'Maine':'ME',
                               'Michigan':'MI',
                               'Minnesota':'MN',
                               'Montana':'MT',
                               'New Mexico':'NM',
                               'New York':'NY',
                               'North Dakota':'ND',
                               'Ohio':'OH',
                               'Texas':'TX',
                               'Vermont':'VT',
                               'Washington':'WA'})

In [49]:
df = df[['Abrv','State','Longitude','Latitude','Border','Date','Measure','Value']]

In [50]:
df.head()

,Abrv,State,Longitude,Latitude,Border,Date,Measure,Value
0,NY,New York,-73.44253,44.99001,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,16377
1,ND,North Dakota,-97.24333,48.96639,US-Canada Border,3/1/2019 0:00,Bus Passengers,1054
2,ND,North Dakota,-98.99457,48.94105,US-Canada Border,3/1/2019 0:00,Personal Vehicle Passengers,509
3,WA,Washington,-122.44316,48.94802,US-Canada Border,3/1/2019 0:00,Pedestrians,79
4,ME,Maine,-67.42955,45.55984,US-Canada Border,3/1/2019 0:00,Pedestrians,3


In [51]:
# Remove non-pedestrian values to shrink file
df.to_csv(Path.joinpath(Path.cwd(),'data','borderCrossing.csv'))